In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install -U pandas-profiling
!pip install sefr_cut

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.7/dist-packages (3.0.0)


In [ ]:
import pandas as pd
import pythainlp
import sefr_cut
import gensim
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

In [ ]:
df = pd.read_csv('CustomerReviews.csv')

In [ ]:
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


Tokenize Words

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n','\n\n', 'ร้าน', '(', ')' , '           ','–', '!', '!!','-','/','+','😆','🤣','"','','%','\u200b','::']
screening_words = stopwords + removed_words

sefr_cut.load_model(engine='ws1000')
def tokenize_with_space(sentence):
  merged = ''
  # words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  words = sefr_cut.tokenize(sentence)
  for word in words[0]:
    if word not in screening_words:
      word = word.rstrip("\n")
      word = word.rstrip("\u200b")
      if word is None or word == ' ' or word == '' or word ==':':
        continue
      else: 
        merged = merged + ',' + word
    elif word is None:
      continue
  return merged[1:]
  
  # return words

loading model.....
Success


In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Review ID         21 non-null     int64  
 1   Restaurant_ID     21 non-null     object 
 2   Restaurant        21 non-null     object 
 3   User              21 non-null     object 
 4   Headline          21 non-null     object 
 5   Review            21 non-null     object 
 6   Rating            20 non-null     float64
 7   Review_tokenized  21 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.4+ KB


In [ ]:
df['Review_tokenized'].iloc[10]


'อัพเดทราคา,ชาบูชิ,ตอน,399,บาท,net,ทาน,1.,15,ชม.,น้ำซุป,เลือก,4,รส,ซูชิลาย,ตา,เทมปุระ,ทาน,เพลิน,ดี,ยืน,ทาน,ไลน์,คาว,บุฟเฟ่ต์,กุ้ง,สด,เบ,เนื้อ,สไลด์,หลัก,อิ่ม,คุ้ม'

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"ลอง,สาขา,ยู,เนี่ยน,มอลล์,รอบ,สาขา,เดอะมอลล์,บา..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,รวมVAT,ทาน,..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"รู้จัก,union,mall,ลอง,กิน,ขึ้นใจ,ชื่อ,ชื่อ,อยู..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้ม,ค่า,สมราคา,บุฟเฟ่,หมู,ผัก,น้ำ,จบ,25..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"จ่าย,ราคา,199,บาท,จ่าย,เงิน,สด,ราคา,น้ำ,VAT 7,..."



Create Dictionary

In [ ]:
# documents = df['Review_tokenized'].to_list()
documents = df['Review_tokenized']
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['1', '4', '560', 'กิน', 'ก็รสชาติ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'จิ้ม', 'ซุป', 'ดี', 'ติม', 'ต้องการ', 'ถาด', 'ถาม', 'น้ำ', 'บริการ', 'บริการรสชาติ', 'บาท', 'ประทับใจ', 'พนักงาน', 'รสชาติ', 'รัก', 'ราคา', 'ร่อย', 'ละกัน', 'สรุป', 'สะอาด', 'สะอ้าน', 'สัน', 'อร่อย', 'อาหาร', 'เคลม', 'เจี้ยว', 'เซ็ท', 'เต้า', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แปลก', 'แฟน', 'โมจิอ', 'ใจ', 'ใส่', 'ไอ', '\u200b ', '\u200bกับ', '\u200bดำ', '\u200bดี', '\u200bพอ', '\u200bภาพ', '\u200bหลากหลาย', '\u200bแต่', '\u200bใหม่', '\u200bไม่', '21', '30', '9', 'CTW', 'Terminal', 'การงดรับ', 'งาน', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ดื่ม', 'ทะยอย', 'ทาน', 'ทำ', 'นาที', 'นึก', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'รายละเอียด', 'ลูกค้า', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่อง', 'เค้า', 'เจอ', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอกาส', 'โอเค', '', '2', '555', '600+', 'กก', 'กรุบ', 'กั้น', 'คน', 'คาว', 'คุโรบู', 'ค่ะ', 'ค่า', 'จาน', 'จืด', 'ฉาก', 'ชอบ', 'ชา', 'ชาบูน้ำดำ', 'ชิ้น', 'ชื่อ', 'ซอยซอส', 'ตก', 'ตะ', 'ตัก', 'ตัว',

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

Topic Modelling

In [ ]:
num_topics = 5
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 100
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 328 ms, sys: 140 µs, total: 329 ms
Wall time: 334 ms


In [ ]:
# pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.050687 -0.030148       1        1  37.937576
3     -0.056323 -0.023294       2        1  30.942346
0     -0.006281  0.023208       3        1  14.930889
4      0.011006  0.033120       4        1  13.611507
2      0.000911 -0.002886       5        1   2.577682, topic_info=        Term       Freq      Total Category  logprob  loglift
11        ดี  28.000000  28.000000  Default  30.0000  30.0000
68       ทาน  24.000000  24.000000  Default  29.0000  29.0000
145    เลือก  19.000000  19.000000  Default  28.0000  28.0000
31     อร่อย  17.000000  17.000000  Default  27.0000  27.0000
32     อาหาร  21.000000  21.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
140     หวาน   0.242983  13.008199   Topic5  -4.9158  -0.3221
63      ชาบู   0.213305   8.696968   Topic5  -5.0460  -0.0497
114      ตัว   0.205584   9.166635   Topic5  -5.0829  -0.1392
21   พนักงาน   0.192359   8.091508   Topic5  -5.1494  -0.0809
132       สด   0.197081  12.270883   Topic5  -5.1252  -0.4731

[327 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
90        2  0.878915         
458       2  0.897565     HALL
459       2  0.897577   ธรรมดา
202       1  0.356668        )
202       4  0.356668        )
...     ...       ...      ...
158       4  0.165210      ไก่
160       2  0.537115       ได
161       2  0.537136      ไผ่
162       2  0.535907      ไม้
46        3  0.505623       ​ 

[301 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 3])

In [ ]:
model.show_topic(3)

[('ทาน', 0.027404325),
 ('น้ำ', 0.024103394),
 ('เนื้อ', 0.022638202),
 ('เลือก', 0.019307017),
 ('อร่อย', 0.018245785),
 ('ซุป', 0.016854124),
 ('บาท', 0.016843596),
 ('ราคา', 0.015618945),
 ('', 0.014608367),
 ('ดี', 0.014080539)]

In [ ]:
# get from most 10 tokens
def convert_to_topic(x):
  topic = ""
  for i in range(len(x)):
    topic += x[i][0]
    if i >= 9:
      break;
  return topic

In [ ]:
# gen. topic name
gen_topic_names = []
for i in range(model.num_topics):
  topic_name = convert_to_topic(model.show_topic(i))
  gen_topic_names.append(topic_name)

print (gen_topic_names)

['ดีกินน้ำเนื้ออร่อยอาหารราคาเลือกทานชาบู', 'ดีกินอาหารเนื้อคนทานกุ้งราคาสดหวาน', 'ดีทานเลือกอร่อยเนื้ออาหารกินคุ้มน้ำราคา', 'ทานน้ำเนื้อเลือกอร่อยซุปบาทราคาดี', 'ดีอาหารทานเลือกสาขาคุ้มเนื้อราคากินบาท']


In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])
df['topic_name'] = df['Review_tokenized'].apply(lambda x: model.show_topic(model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0]))
df['topic_name2'] = df['topic_name'].apply(lambda x:convert_to_topic(x))


In [ ]:
pd.set_option('display.max_columns', None)
df[['Restaurant', 'Review', 'topics', 'topic_name2','score']].head(30)

,Restaurant,Review,topics,topic_name2,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,0,ดีกินน้ำเนื้ออร่อยอาหารราคาเลือกทานชาบู,0.998378
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,4,ดีอาหารทานเลือกสาขาคุ้มเนื้อราคากินบาท,0.997422
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,3,ทานน้ำเนื้อเลือกอร่อยซุปบาทราคาดี,0.999307
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,0,ดีกินน้ำเนื้ออร่อยอาหารราคาเลือกทานชาบู,0.996225
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,0,ดีกินน้ำเนื้ออร่อยอาหารราคาเลือกทานชาบู,0.997093
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,1,ดีกินอาหารเนื้อคนทานกุ้งราคาสดหวาน,0.996982
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,1,ดีกินอาหารเนื้อคนทานกุ้งราคาสดหวาน,0.998044
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,0,ดีกินน้ำเนื้ออร่อยอาหารราคาเลือกทานชาบู,0.997859
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,4,ดีอาหารทานเลือกสาขาคุ้มเนื้อราคากินบาท,0.996516
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,1,ดีกินอาหารเนื้อคนทานกุ้งราคาสดหวาน,0.997923


#Interpretation:


#Topic :

Topic 0 ดี กิน น้ำ เนื้อ อร่อย อาหาร ราคา เลือกทาน ชาบู #ชาบู

Topic 1 ดี กิน อาหาร เนื้อ คนทานกุ้ง ราคา สด หวาน #กุ้ง สด หวาน

Topic 2 ดี ทาน เลือก อร่อย เนื้อ อาหาร กิน คุ้ม น้ำ ราคา

Topic 3 ทาน น้ำ เนื้อ เลือก อร่อย ซุป บาท ราคาดี 

Topic 4 ดี อาหาร ทาน เลือก สาขา คุ้ม เนื้อ ราคา กิน บาท #สาขา

#keywords ที่ปรากฎไม่ซ้ำ Topic อื่นๆ

#Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ
From 8 reviews; classify as topic 0=4, topic 1=2, topic 3=1, and topic 4=1

#Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G
From 8 reviews; classify as topic 1=3, topic 2=1, topic 3=2, and topic 4=2

#ข้าน้อยขอชาบู
From 5 reviews; classify as topic 1=2 and topic 3=3